In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler, get_openai_callback
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

set_llm_cache(SQLiteCache())

chat = ChatOpenAI(streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

with get_openai_callback() as usage:
    chat.predict("What's the recipe for sangria?")
    print(usage)

Tokens Used: 355
	Prompt Tokens: 15
	Completion Tokens: 340
Successful Requests: 1
Total Cost (USD): $0.0007025


In [10]:
# Predict

prompt = PromptTemplate.from_template(
    "What is the advantage of {frontend_stack}?"
).format(frontend_stack="Svelte")
chat.predict(prompt)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a frontend expert. Also, you only reply in {language}"),
        ("ai", "{tone}"),
        ("human", "What is the advantage of {frontend_stack}? Also, what's your name?"),
    ]
).format_messages(language="Korean", tone="안녕하세요!", frontend_stack="Next.js")
chat.predict_messages(prompt)

Svelte has several advantages over other JavaScript frameworks and libraries:

1. Performance: Svelte compiles the code at build time, which means that the resulting bundle sent to the browser is highly optimized and requires minimal runtime overhead. This results in faster loading times and better overall performance compared to frameworks that need to execute code at runtime.

2. Small bundle size: Svelte generates highly efficient and minimal JavaScript code, resulting in smaller bundle sizes. This means faster downloads, reduced bandwidth usage, and improved user experience, especially on mobile devices and in areas with slower internet connections.

3. Easy learning curve: Svelte has a simple and intuitive syntax that is easy to learn and understand. It does not have a steep learning curve like some other frameworks, making it an attractive option for developers, particularly those who are new to front-end development.

4. Reactive programming: Svelte uses a reactive programming p

AIMessageChunk(content='안녕하세요! 저는 한국어로 답변하는 AI 어시스턴트입니다. Next.js의 장점은 다음과 같습니다:\n\n1. 서버 사이드 렌더링 (SSR): Next.js는 서버 사이드 렌더링을 기본적으로 지원하여 초기 페이지 로딩 속도를 개선하고 SEO에 뛰어난 성능을 제공합니다.\n\n2. 정적 사이트 생성 (SSG): Next.js는 정적 사이트 생성을 지원하여 사전 렌더링을 통해 성능을 향상시킵니다. 정적 사이트 생성은 동적인 데이터가 자주 변경되지 않는 경우에 유용합니다.\n\n3. 자동 코드 분할 (Automatic Code Splitting): Next.js는 페이지 단위로 자동으로 코드를 분할하여 필요한 부분만을 로드하므로 초기 페이지 로딩 속도가 향상됩니다.\n\n4. 간편한 라우팅: Next.js는 파일 시스템 기반의 라우팅을 제공하여 라우팅 설정이 간단합니다. 파일 이름에 따라 페이지 경로가 자동으로 생성되므로 개발자가 별도의 라우팅 설정을 할 필요가 없습니다.\n\n5. 풍부한 생태계: Next.js는 프론트엔드 개발에 필요한 다양한 기능과 라이브러리를 포함한 풍부한 생태계를 제공합니다. 예를 들어, CSS-in-JS 라이브러리인 Styled Components나 상태 관리 라이브러리인 Redux와의 통합이 간편합니다.\n\n저는 AI 어시스턴트이므로 개인 이름은 없습니다. 어떤 점을 도움 드릴까요?')

In [11]:
# Output Parser

from langchain.schema import BaseOutputParser


class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        return list(map(str.strip, text.split(",")))


parser = CommaOutputParser()
parser.parse(" Hello, how,are,you ")

['Hello', 'how', 'are', 'you']

In [12]:
# Langchain Expression Language

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do NOT reply with anything else",
        ),
        ("human", "{question}"),
    ]
)
chain = prompt | chat | parser
chain.invoke(
    {"max_items": 5, "question": "Interesting things about frontend development"}
)

html, css, javascript, responsive design, user experience

['html', 'css', 'javascript', 'responsive design', 'user experience']

In [13]:
# Chaining Chains

chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a world-class international chef. You create easy to follow recipes for any type of cuisine with easy to find ingredients.",
        ),
        ("human", "I want to cook {cuisine} food."),
    ]
)
chef_chain = chef_prompt | chat

veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a vegetarian chef specialized on making traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to recipe it.",
        ),
        ("human", "{recipe}"),
    ]
)
veg_chef_chain = veg_chef_prompt | chat

chain = {"recipe": chef_chain} | veg_chef_chain
chain.invoke({"cuisine": "indian"})

Great choice! Indian cuisine is full of vibrant flavors and aromatic spices. Let's start with a classic Indian dish called Butter Chicken. Here's a simple and delicious recipe for you:

Ingredients:
- 500g boneless chicken, cut into bite-sized pieces
- 2 tablespoons butter
- 1 large onion, finely chopped
- 2 cloves of garlic, minced
- 1-inch piece of ginger, grated
- 2 teaspoons garam masala
- 1 teaspoon turmeric powder
- 1 teaspoon chili powder (adjust to your spice preference)
- 1 cup tomato puree
- 1/2 cup heavy cream
- Salt to taste
- Fresh cilantro leaves for garnish

Instructions:
1. Heat the butter in a large pan over medium heat. Add the chopped onions and sauté until golden brown.
2. Add the minced garlic and grated ginger to the pan. Cook for another minute until fragrant.
3. In a small bowl, mix the garam masala, turmeric powder, and chili powder. Add this spice mixture to the pan and cook for a minute to release the flavors.
4. Add the chicken pieces to the pan and cook unt

AIMessageChunk(content="Great choice! Butter Chicken is a delicious and popular Indian dish. To make it vegetarian, we'll need to replace the chicken with a suitable alternative. Here's a vegetarian version of the recipe:\n\nIngredients:\n- 500g paneer (Indian cottage cheese), cut into bite-sized pieces\n- 2 tablespoons butter (or vegan butter for a vegan version)\n- 1 large onion, finely chopped\n- 2 cloves of garlic, minced\n- 1-inch piece of ginger, grated\n- 2 teaspoons garam masala\n- 1 teaspoon turmeric powder\n- 1 teaspoon chili powder (adjust to your spice preference)\n- 1 cup tomato puree\n- 1/2 cup heavy cream (or coconut cream for a vegan version)\n- Salt to taste\n- Fresh cilantro leaves for garnish\n\nInstructions:\n1. Heat the butter in a large pan over medium heat. Add the chopped onions and sauté until golden brown.\n2. Add the minced garlic and grated ginger to the pan. Cook for another minute until fragrant.\n3. In a small bowl, mix the garam masala, turmeric powder, 

In [14]:
# FewShotPromptTemplate, Example Selector

from langchain.prompts.few_shot import (
    FewShotPromptTemplate,
    FewShotChatMessagePromptTemplate,
)
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
    },
]


class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]


template = "Human: {question}\nAI: {answer}"
prompt = PromptTemplate.from_template(template)
selector = RandomExampleSelector(examples=examples)
prompt = FewShotPromptTemplate(
    example_prompt=prompt,
    example_selector=selector,
    suffix="Human: {question}",
    input_variables=["question"],
)

chain = prompt | chat
chain.invoke({"question": "I wonder about Sweden"})

AI: 
Here is what I know about Sweden:
Capital: Stockholm
Language: Swedish
Food: Meatballs, herring, and lingonberries
Currency: Swedish Krona
Famous for: ABBA, IKEA, Volvo, and beautiful natural landscapes such as the Northern Lights and the archipelago.

AIMessageChunk(content='AI: \nHere is what I know about Sweden:\nCapital: Stockholm\nLanguage: Swedish\nFood: Meatballs, herring, and lingonberries\nCurrency: Swedish Krona\nFamous for: ABBA, IKEA, Volvo, and beautiful natural landscapes such as the Northern Lights and the archipelago.')

In [15]:
# FewShotChatMessagePromptTemplate

examples = [
    {
        "country": "France",
        "answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
    },
    {
        "country": "Italy",
        "answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
    },
    {
        "country": "Greece",
        "answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
    },
]

prompt = ChatPromptTemplate.from_messages(
    [("human", "What do you know about {country}?"), ("ai", "{answer}")]
)
prompt = FewShotChatMessagePromptTemplate(
    example_prompt=prompt,
    examples=examples,
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert, you give short answers"),
        prompt,
        ("human", "What do you know about {country}?"),
    ]
)

chain = prompt | chat
chain.invoke({"country": "Sweden"})

I know this:
Capital: Stockholm
Language: Swedish
Food: Meatballs and Herring
Currency: Swedish Krona

AIMessageChunk(content='I know this:\nCapital: Stockholm\nLanguage: Swedish\nFood: Meatballs and Herring\nCurrency: Swedish Krona')

In [16]:
# Prompt Serialization

from langchain.prompts import load_prompt

prompt = load_prompt("./prompt.yaml")
chain = prompt | chat
chain.invoke({"country": "South Korea"})

The capital of South Korea is Seoul.

AIMessageChunk(content='The capital of South Korea is Seoul.')

In [17]:
# Prompt Composition

from langchain.prompts.pipeline import PipelinePromptTemplate

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]
full_prompt = PipelinePromptTemplate(final_prompt=final, pipeline_prompts=prompts)

chain = full_prompt | chat
chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)

Arrrgh, me heartie! Me favorite food be none other than a hearty plate o' seafood! I can't resist the taste o' fresh fish, lobster, and shrimp straight from the depths o' the sea. And don't ye forget about me trusty bottle o' rum to wash it all down! Arrrr!

AIMessageChunk(content="Arrrgh, me heartie! Me favorite food be none other than a hearty plate o' seafood! I can't resist the taste o' fresh fish, lobster, and shrimp straight from the depths o' the sea. And don't ye forget about me trusty bottle o' rum to wash it all down! Arrrr!")

In [15]:
# Model Serialization

from langchain.llms.openai import OpenAI
from langchain.llms.loading import load_llm

chat = OpenAI(temperature=0.1, max_tokens=450, model="gpt-3.5-turbo-16k")
chat.save("model.json")

chat = load_llm("model.json")
chat.predict("Who are you?")

/Users/jihyun/Desktop/gpt/env/lib/python3.11/site-packages/langchain/llms/openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/jihyun/Desktop/gpt/env/lib/python3.11/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


'I am an AI language model developed by OpenAI. I am designed to assist with various tasks and provide information on a wide range of topics. How can I assist you today?'